In [13]:
import io
import os
import pandas as pd
from google.colab import drive
import re
from google.colab import files
import datetime
from pytz import timezone


input_file_base_url = 'My Files/Papa/'        #Please Give slash at end

input_text_file_name = 'afos.txt' 

output_file = str(datetime.datetime.now(timezone('US/Central')).strftime("%Y-%m-%d %H:%M:%S")) +'.csv'

output_row = ['TEMP', 'DEWPT', 'RH', 'WIND DIR', 'WIND SPD', 'CLOUDS', 'POP 12HR', 'QPF 12HR', 'SNOW 12HR', 'MAX HEAT']

weekDays = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']


def temp_process_first_half(temp_half, gen_info):
  output_line_2 = [[]]
  output_column = ['']
  min_max_index = []
  is_min_max_found = False

  date_line = gen_info[-1]                                                      #date at last row             
  date_line = date_line[:-1]            
  date_column_name = date_line[:4]                                              #first output_col
  date_line = date_line[4:]

  dates = date_line.split('   ')
  dates = [x for i,x in enumerate(dates) if x]

  gen_info = gen_info[:-2]

  with open(output_file,'a') as fd:
    fd.write('\n')
    for info in gen_info:
      fd.write(info)                                                            # writing area name and general info
    fd.write('\n\n')

  del temp_half[2]
  is_max_min = False

  ###temp_half_fixed = temp_half[:12]                                           # until Snow 12HR, if user want more change to 'temp_half[:len(temp_half_fixed)]'
  temp_half_fixed = temp_half
  temp_half_dynamic = temp_half[12:]                                            # to find max_heat only (lines after snow 12HR)

  line_count = -1

  for temp_line in temp_half_fixed:
    temp_col_name = temp_line[:13]                                              # in each line 13 index for column name (data starts after 13 index)
    line_count = line_count + 1

    temp_line = temp_line[13:]
    temp_line = temp_chunk_str(temp_line,3)                                     # the puzzle was each data consists of 3 character, so spliting each 3

    if len(temp_line) > 0:
      del temp_line[len(temp_line)-1]                                           # deleting last index which was '\n'

    if temp_col_name.strip().upper() == 'MAX/MIN' or temp_col_name.strip().upper() == 'MIN/MAX':
      if temp_col_name.strip().upper() == 'MAX/MIN':                            # depending on 'MAX/MIN' the date changes
        is_max_min = True
      else:
        is_max_min = False

      min_max_values = temp_line
      is_min_max_found = True

      for index, val in enumerate(min_max_values):                              # getting only the index which has min/max values
        if val != '   ':
          min_max_index.append(index)
      
    if is_min_max_found and temp_col_name.strip().upper() == 'QPF 12HR':        # if it's 'QPF 12HR' then it has float input like '0.05', so 4 index
      for idx in min_max_index:
        if idx != 0:
          temp_line[idx] = temp_line[idx-1] + temp_line[idx]                    # as 4 index, taking index from previous which is fine
          temp_line[idx-1] = ' '                                                # it show up only when min/max index, so removing previous one

    if is_min_max_found and temp_col_name.strip().upper() == 'SNOW 12HR':       # if it's 'SNOW 12HR' then it has an input like '00-05', so 5 index 
      for idx in min_max_index:
        if idx != 0:
          temp_line[idx] = temp_line[idx-1] + temp_line[idx]                    # as it shows min/max index just like previous, taking 2 chunks of 3 index = 6 index
          temp_line[idx-1] = ' '
                                                                                # if user want specific colum, check 'temp_col_name' before pushing to output_line
    if line_count <= 3:                                                         # 2 time and min max roo                                                                            
      output_line_2.append(temp_line)                                           # inserting to output_line
      output_column.append(temp_col_name.strip())
    else:
      if temp_col_name.strip().upper() in output_row:
        output_line_2.append(temp_line)                                           # inserting to output_line
        output_column.append(temp_col_name.strip())

  
  output_line_2 = output_line_2[:3] + [''] + output_line_2[3:]                  # inserting an empty list after date
  output_column = output_column[:3] + [''] + output_column[3:]                  # inserting an empty column after date

  len_each_line = len(output_line_2[1])
  output_line_2[0] = [''] * len_each_line                                       # for inserting week day, initializing with empty string according to next line

  first_date = ''
  if is_max_min:                                                                # if max/min taking first day
    if len(dates)>0:                                          
      first_date = dates[0].strip().split(' ')[0]                               # as user only wants the week day, not date
  else:
    if len(dates)>1:
      first_date = dates[1].strip().split(' ')[0]                               # if min/max  taking second day, watch data

  first_date_index = 0

  for index, days in enumerate(weekDays):
    if first_date.strip().upper() == days.upper():
      first_date_index = index                                                  # saving the first week day only
      break

  newDay_index = first_date_index                                               # first day getting from data, next inputing from code

  if is_max_min:                                                                # if max/min present it gets first day one time
    for index, value in enumerate(min_max_index):                               # then second date 2 time, third date 2 time
      if index%2==1:
        newDay_index = newDay_index+1
      if len(output_line_2[0])>value and len(weekDays)>newDay_index:
        output_line_2[0][value]=weekDays[newDay_index]
  else:
    for index, value in enumerate(min_max_index):
      if index>0 and index%2==0:                                                # if min/max present it gets first day twice
        newDay_index = newDay_index+1                                           # then second date 2 time, third date 2 time
      if len(output_line_2[0])>value and len(weekDays)>newDay_index:
        output_line_2[0][value]=weekDays[newDay_index]

  df = pd.DataFrame(output_line_2, index=output_column)                         # writing to csv file append mode
  df.to_csv(output_file, mode='a', header=False)
  
  with open(output_file,'a') as fd:
    fd.write('\n\n')

  return newDay_index                                                           # returning week day index to use in next half part


def divide_zone(temp_zone_to_divide, gen_info):

  for index, line_1 in enumerate(temp_zone_to_divide):
      if index != len(temp_zone_to_divide)-1:                                   # diving if it gets two new line
        if temp_zone_to_divide[index] == '\n' and temp_zone_to_divide[index+1] == '\n':
          first_half = temp_zone_to_divide[:index]
          second_half = temp_zone_to_divide[index+2:]
          break
  
  newDay_index = temp_process_first_half(first_half, gen_info)
  temp_process_second_half(second_half, newDay_index)


def get_zone_general_information(temp_zone_info):
  zone_general_info = temp_zone_info[:6]                                        # date for the first 3 is present here


def process_zone(temp_zone):
  if temp_zone[0] == '\n':
    temp_zone = temp_zone[1:]

  get_zone_general_information(temp_zone)
  gen_info = temp_zone[:6]
  temp_zone = temp_zone[6:]                                                     # removing general info
  
  divide_zone(temp_zone, gen_info)

  
def divide_zones(lines):
  end_index = 0
  zones = [[]]

  for line in lines:
    end_index = end_index + 1
    if line == '$$\n':
      zone = lines[:end_index]
      lines = lines[end_index:]
      end_index = 0
      zones.append(zone)
  
  zones = zones[1:]

  for zone in zones:
    process_zone(zone)


def get_general_information(lines):
  general_information = lines[:9]


def start_process(lines):

  get_general_information(lines)
  lines = lines[9:]                                                             #removing lines which has general information
  divide_zones(lines)


def temp_chunk_str(str, chunk_size):
   return [str[i:i+chunk_size] for i in range(0, len(str), chunk_size)]


def temp_process_second_half(temp_second_half, newDay_index):

  with open(output_file,'a') as fd:
    fd.write('\n')
  
  output_column = ['']
  output_line_2 = [[]]
  min_max_index = []
  is_min_max_found = False

  date_line = temp_second_half[0]
  date_line = date_line[4:].strip()

  dates = date_line.split('  ')
  dates = [x for i,x in enumerate(dates) if x]

  temp_second_half_fixed = temp_second_half[1:11]
  del temp_second_half_fixed[2]

  find_all_gap = []
  is_gap_found = False
  
  line_count = 1
  is_min_max = False

  for temp_line in temp_second_half_fixed:
    temp_col_name = temp_line[:13]
    output_column.append(temp_col_name.strip())

    temp_line = temp_line[13:]

    if line_count == 1:                                                         # The puzzle was if there was two gap in first date column remove both
      find_all_gap = [m.start() for m in re.finditer('   ', temp_line)]
      line_count = 2
      is_gap_found = True
    
    if is_gap_found:                                                            # removing gap, but the string gets smaller so gap reduce through count
      count = 0
      for gap in find_all_gap:
        gap = gap - count*2
        temp_line = temp_line[:gap] + temp_line[gap+2:]
        count = count + 1

    temp_line = temp_chunk_str(temp_line,3)
    del temp_line[len(temp_line)-1]

    if temp_col_name.strip().upper() == 'MAX/MIN' or temp_col_name.strip().upper() == 'MIN/MAX':
      if temp_col_name.strip().upper() == 'MIN/MAX':
        is_min_max = True                                                       # read for first half
      else:
        is_min_max = False

      min_max_values = temp_line
      is_min_max_found = True

      for index, val in enumerate(min_max_values):
        if val != '   ':
          min_max_index.append(index)
    
    if temp_col_name.strip().upper() == 'WIND CHAR':
      wind_char_values = temp_line

      for index, wind_char_val in enumerate(wind_char_values):                  # according to user requirements
        if wind_char_val.strip().upper() == 'LT':
          wind_char_values[index] = '5'
        elif wind_char_val.strip().upper() == 'GN':
          wind_char_values[index] = '10'
        elif wind_char_val.strip().upper() == 'BZ':
          wind_char_values[index] = '20'
        elif wind_char_val.strip().upper() == 'WY':
          wind_char_values[index] = '30'
        elif wind_char_val.strip().upper() == 'VW':
          wind_char_values[index] = '35'
        elif wind_char_val.strip().upper() == 'SD':
          wind_char_values[index] = '40'

      temp_line = wind_char_values

    output_line_2.append(temp_line)
  
  output_line_2 = output_line_2[:3] + [''] + output_line_2[3:]                  # read for first half
  output_column = output_column[:3] + [''] + output_column[3:]
  
  len_each_line = len(output_line_2[1])
  output_line_2[0] = [''] * len_each_line

  day_index = newDay_index

  if is_min_max:
    day_index = day_index + 1                                                   # read for first half
    for index, value in enumerate(min_max_index):
      if index>0 and index%2==0:
        day_index = day_index+1
      if len(output_line_2[0])>value and len(weekDays)>newDay_index:
        output_line_2[0][value]=weekDays[day_index]
  else:
    for index, value in enumerate(min_max_index):
      if index%2==1:
        day_index = day_index+1
      if len(output_line_2[0])>value and len(weekDays)>newDay_index:
        output_line_2[0][value]=weekDays[day_index]

  df = pd.DataFrame(output_line_2, index=output_column)
  df.to_csv(output_file, mode='a', header=False)

  with open(output_file,'a') as fd:
    fd.write('\n\n\n')


try:
  drive.mount('/drive')
  file1 = open('/drive/My Drive/' + input_file_base_url + input_text_file_name,'r')
  lines = file1.readlines()

  try:
    start_process(lines)
    files.download(output_file)
  except:
    code_error = 'Code/Data error! Contact Husnul_' + output_file
    with open(code_error,'a') as fd:
      fd.write("""\nSorry !! Seems like there is a problem with the data or format.
                \n Please contact me in upWork (Husnul Abid).
                \n My account url : https://www.upwork.com/freelancers/~0128f7ecfc4bb12216
            """)
    files.download(code_error)

except:
  file_error = 'File error! Run again_' + output_file
  with open(file_error,'a') as fd:
    fd.write("""\nSorry !! We could not find your file. The soluiton can be
                \n1. Please make sure there is a file and the name is afos.txt.
                \n2. It should be on right location which is: /drive/My Drive/""" + input_file_base_url +
                """\n\n3. Please wait 4/5 second and REFRESH after uploading the file
                \n4. If there is an existing afos.txt Do not replace it. Delete and upload a new one with same name
                \n5. Authorize with the google account in which you have the file.
            """)
    files.download(file_error)


Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>